<a href="https://colab.research.google.com/github/ikhwanafif05/Credit-Risk-Modelling-Tools/blob/main/Portfolio_Stress_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

# ==========================================
# COMMERCIAL WEAPON: PORTFOLIO STRESS TESTER
# Author: Ikhwan Afif
# Purpose: Simulate ICAAP Interest Rate Shocks on Loan Portfolios
# ==========================================

def run_stress_test(portfolio_value, avg_interest_rate, shock_bps):
    """
    Runs a sensitivity analysis on a loan portfolio based on interest rate shocks.

    Parameters:
    - portfolio_value: Total Exposure at Default (EAD)
    - avg_interest_rate: Current weighted average interest rate (decimal)
    - shock_bps: Basis point shock (e.g., 50 for +0.50%, 200 for +2.00%)
    """

    print(f"--- RUNNING STRESS TEST: +{shock_bps} BPS SHOCK ---")

    # Convert basis points to percentage
    shock_pct = shock_bps / 10000
    new_rate = avg_interest_rate + shock_pct

    # Calculate Impact on Debt Service
    base_interest_income = portfolio_value * avg_interest_rate
    stressed_interest_income = portfolio_value * new_rate

    # The "Pain" Calculation: Additional Cash Flow required from borrowers
    additional_burden = stressed_interest_income - base_interest_income

    print(f"Portfolio Size: RM {portfolio_value:,.0f}")
    print(f"Base Rate: {avg_interest_rate:.2%} | New Rate: {new_rate:.2%}")
    print(f"Borrower Additional Burden: RM {additional_burden:,.0f} per year")

    # Risk Logic: Mock elasticity model
    # Assumption: For every 1% rate hike, Probability of Default (PD) increases by 15%
    pd_increase_factor = 1 + (shock_pct * 15)
    print(f"Projected PD Increase Factor: {pd_increase_factor:.2f}x")
    print("-" * 40)

if __name__ == "__main__":
    # EXECUTE: Simulate ICAAP Scenarios (50bps and 200bps)
    run_stress_test(portfolio_value=100000000, avg_interest_rate=0.045, shock_bps=50)
    run_stress_test(portfolio_value=100000000, avg_interest_rate=0.045, shock_bps=200)

--- RUNNING STRESS TEST: +50 BPS SHOCK ---
Portfolio Size: RM 100,000,000
Base Rate: 4.50% | New Rate: 5.00%
Borrower Additional Burden: RM 500,000 per year
Projected PD Increase Factor: 1.07x
----------------------------------------
--- RUNNING STRESS TEST: +200 BPS SHOCK ---
Portfolio Size: RM 100,000,000
Base Rate: 4.50% | New Rate: 6.50%
Borrower Additional Burden: RM 2,000,000 per year
Projected PD Increase Factor: 1.30x
----------------------------------------
